In [1]:
import lightgbm as lgb
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy import sparse 
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import chi2, SelectPercentile,SelectKBest
import gc
import time
from sklearn.feature_selection import chi2, SelectPercentile

In [2]:
from scipy.sparse import csr_matrix
def save_csr(matrix,file_prefix='csr'):
    data_file = file_prefix + '_data.npy'
    indices_file = file_prefix + '_indices.npy'
    indptr_file = file_prefix + '_indptr.npy'
    np.save(data_file,matrix.data)
    print('data save')
    np.save(indices_file,matrix.indices)
    print('indices save')
    np.save(indptr_file,matrix.indptr)
    print('indptr save')
    print('save done!')
    
def load_csr(file_prefix='csr'):
    data_file = file_prefix + '_data.npy'
    indices_file = file_prefix + '_indices.npy'
    indptr_file = file_prefix + '_indptr.npy'
    data = np.load(data_file)
    print('data load')
    indices = np.load(indices_file)
    print('indices load')
    indptr = np.load(indptr_file)
    print('indptr load')
    csr = csr_matrix((data, indices, indptr))
    return csr

In [3]:
train_csr = load_csr('../../data/features/xgb/train_csr')
predict_csr = load_csr('../../data/features/xgb/predict_csr')
train_y = np.load('../../data/features/xgb/train_y.npy')
predicted_file = pd.read_pickle('../../data/features/xgb/predict_id.pkl')

data load
indices load
indptr load
data load
indices load
indptr load


# Load More features

In [5]:
X_fe = pd.read_pickle('../../data/features/xgb/new_feats_838.pkl')
print(X_fe.shape)
X_ = X_fe.copy()



(3072915, 838)


In [6]:
cut_bin = 30
for col in tqdm(X_fe.columns):
        X_[col] = pd.cut(X_fe[col], bins=cut_bin, labels=False, retbins=True, right=False)[0]  

100%|██████████| 838/838 [11:43<00:00,  1.19it/s]


# Append more data from SVD

In [6]:
# train_csr_f = load_csr('../../data/features/xgb/train_csr_2500')
# predict_csr_f = load_csr('../../data/features/xgb/predict_csr_2500')
# train_instance = pd.read_pickle('../../data/features/xgb/train_noclean_bytime.pkl')[['instance_id']].copy()
# train_y = np.load('../../data/features/xgb/train_y_f620_b70_col17k.npy')

# X_u = pd.read_pickle('../../data/features/agg/svd_utag_adid_cnt.pkl')
# X_u = train_instance.merge(X_u,on='instance_id',how='inner')
# X_u.drop('instance_id',inplace=True,axis=1)

# cut_bin = 30
# # selected_col = [col for col in X_u.columns if 'utagsIndividual_svd' not in col]
# # X_u = X_u[selected_col]
# for col in tqdm(X_u.columns):
#         X_u[col] = pd.cut(X_u[col], bins=cut_bin, labels=False, retbins=True, right=False)[0]  
        
# print(train_csr_f.shape)
# train_csr_f = sparse.hstack(
#     (sparse.csr_matrix(X_u.iloc[:train_csr_f.shape[0]]), train_csr_f), 'csr').astype(
#     'float32')
# predict_csr_f = sparse.hstack(
#     (sparse.csr_matrix(X_u.iloc[train_csr_f.shape[0]:]), predict_csr_f), 'csr').astype('float32')
# print(train_csr_f.shape)
        
        

In [8]:
# train_csr_f = sparse.hstack(
#     (sparse.csr_matrix(X_.iloc[:train_csr.shape[0]]), train_csr_10k), 'csr').astype(
#     'float32')
# predict_csr_f = sparse.hstack(
#     (sparse.csr_matrix(X_.iloc[train_csr.shape[0]:]), predict_csr_10k), 'csr').astype('float32')



train_csr_f = sparse.hstack(
    (sparse.csr_matrix(X_.iloc[:train_csr.shape[0]]), train_csr), 'csr').astype(
    'float32')
predict_csr_f = sparse.hstack(
    (sparse.csr_matrix(X_.iloc[train_csr.shape[0]:]), predict_csr), 'csr').astype('float32')

print(train_csr_f.shape)
print(predict_csr_f.shape)



(2992639, 34740)
(80276, 34740)


In [9]:
# feature_select = SelectKBest(chi2, percentile=10000)
# feature_select.fit(train_csr, train_y)
# train_csr = feature_select.transform(train_csr)
# predict_csr = feature_select.transform(predict_csr)
# print('feature select')
# print(train_csr.shape)

# Generate sample

In [10]:
# sample_size = 400000
# sample_times = 2
# sample_all = pd.DataFrame({'data':range(train_csr.shape[0])})
# sample_index_list = []
# samples = []
# samples_y = []
# for i in tqdm(range(sample_times)):
#     sample_index_list.append(list(sample_all.sample(sample_size).index))
#     samples.append(train_csr[sample_index_list[i]])
#     samples_y.append(train_y[sample_index_list[i]])


# Define lgb model

In [15]:
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=32, reg_alpha=0, reg_lambda=0.1,
    max_depth=-1, n_estimators=400, objective='binary',
    subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.05, random_state=1001, n_jobs=16
)


In [10]:
def test_on_sample(train_csr_f,train_y_f,lgb_model,n_fold = 5,seed=None,first=2500):
    if seed is None:
        seed = np.random.randint(1000)
    cv_list = []
    skf = StratifiedKFold(n_splits=n_fold, random_state=seed, shuffle=True)
    best_score = []


    feature_importance_tmp_split = []
    feature_importance_tmp_gain = []
    df_fea = pd.DataFrame()

    t1 = time.time()
    for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y_f)):
        lgb_model.fit(train_csr_f[train_index], train_y_f[train_index],
                      eval_set=[(train_csr_f[train_index], train_y_f[train_index]),
                                (train_csr_f[test_index], train_y_f[test_index])], early_stopping_rounds=200,verbose=200)
        best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
        print(best_score)
        print('cv:',np.mean(best_score))
        test_pred = lgb_model.predict_proba(predict_csr_f, ntree_limit=lgb_model.best_iteration_)[:, 1]
        print('test mean:', test_pred.mean())
        feature_importance_tmp_split.append(lgb_model.booster_.feature_importance(importance_type='split'))
        feature_importance_tmp_gain.append(lgb_model.booster_.feature_importance(importance_type='gain'))

    df_fea['col_name'] = lgb_model.booster_.feature_name()
    df_fea['split'] = np.mean(feature_importance_tmp_split,axis=0)
    df_fea['gain'] = np.mean(feature_importance_tmp_gain,axis=0)

    cv_list.append(np.mean(best_score))
    t2 = time.time()
    print('time spend: {} s'.format(t2-t1))
    print('after dropping...')


    split_index = list(df_fea[df_fea.split!=0].index)
    gain_index = list(df_fea[df_fea.gain!=0].index)
    #union
#     retain_index = list(set(split_index+gain_index))
    #intersection
#     retain_index = list(set(split_index).intersection(set(gain_index)))
    if first is None:
        first = 500
    split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
    gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
    retain_index = list(set(split_index).intersection(set(gain_index)))
    print(len(retain_index))

    best_scor2 = []
    t1 = time.time()
    for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y_f)):
        lgb_model.fit(train_csr_f[train_index][:,retain_index], train_y_f[train_index],
                      eval_set=[(train_csr_f[train_index][:,retain_index], train_y_f[train_index]),
                                (train_csr_f[test_index][:,retain_index], train_y_f[test_index])], early_stopping_rounds=200,verbose=200)
        best_scor2.append(lgb_model.best_score_['valid_1']['binary_logloss'])
        print(best_scor2)
        print('cv:',np.mean(best_scor2))
        #####################
        test_pred = lgb_model.predict_proba(predict_csr_f[:,retain_index], num_iteration=lgb_model.best_iteration_)[:, 1]
        print('test mean:', test_pred.mean())
        #####################
    cv_list.append(np.mean(best_scor2))
    drop = False
    if cv_list[-1] < cv_list[-2]:
        print('dropping successfully!')
        print('cv drop:{}'.format(cv_list[-2]-cv_list[-1]))
        drop = True
    else:
        print('dropping failed!!!!!!')
        print('cv increase:{}'.format(cv_list[-1]-cv_list[-2]))
    t2 = time.time()
    print('time spend: {} s'.format(t2-t1))
    return df_fea, retain_index, drop

In [11]:
def verify_on_sample(train_csr_f,train_y_f,lgb_model,retain_index,n_fold = 3,seed=None,cv_list=[]):
    if seed is None:
        seed = np.random.randint(1000)

    skf = StratifiedKFold(n_splits=n_fold, random_state=seed, shuffle=True)
    best_score = []
    
    if len(cv_list) == 0:

        t1 = time.time()
        for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y_f)):
            lgb_model.fit(train_csr_f[train_index], train_y_f[train_index],
                          eval_set=[(train_csr_f[train_index], train_y_f[train_index]),
                                    (train_csr_f[test_index], train_y_f[test_index])], early_stopping_rounds=100,verbose=200)
            best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
            print(best_score)
            print('cv:',np.mean(best_score))




        cv_list.append(np.mean(best_score))
        t2 = time.time()
        print('time spend: {} s'.format(t2-t1))
        print('after dropping...')

        print(len(retain_index))

    best_scor2 = []
    t1 = time.time()
    for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y_f)):
        lgb_model.fit(train_csr_f[train_index][:,retain_index], train_y_f[train_index],
                      eval_set=[(train_csr_f[train_index][:,retain_index], train_y_f[train_index]),
                                (train_csr_f[test_index][:,retain_index], train_y_f[test_index])], early_stopping_rounds=100,verbose=200)
        best_scor2.append(lgb_model.best_score_['valid_1']['binary_logloss'])
        print(best_scor2)
        print('cv:',np.mean(best_scor2))
    cv_list.append(np.mean(best_scor2))
    t2 = time.time()
    print('time spend: {} s'.format(t2-t1))

    if cv_list[-1] < cv_list[-2]:
        print('dropping successfully!')
        print('cv drop:{}'.format(cv_list[-2]-cv_list[-1]))
        return True
    else:
        print('dropping failed!!!!!!')
        print('cv increase:{}'.format(cv_list[-1]-cv_list[-2]))
        return False
    
    

In [ ]:
gc.collect()
df_fea, retain_index, ifdrop = test_on_sample(train_csr_f,train_y,lgb_model,n_fold = 5,seed=19,first=2500)

Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_logloss: 0.416003	valid_1's binary_logloss: 0.415773
[400]	valid_0's binary_logloss: 0.413798	valid_1's binary_logloss: 0.41469
Did not meet early stopping. Best iteration is:
[400]	valid_0's binary_logloss: 0.413798	valid_1's binary_logloss: 0.41469
[0.41468987784507]
cv: 0.41468987784507
test mean: 0.21385778288287302
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_logloss: 0.415734	valid_1's binary_logloss: 0.416863
[400]	valid_0's binary_logloss: 0.413504	valid_1's binary_logloss: 0.415771
Did not meet early stopping. Best iteration is:
[400]	valid_0's binary_logloss: 0.413504	valid_1's binary_logloss: 0.415771
[0.41468987784507, 0.41577065575283384]
cv: 0.4152302667989519
test mean: 0.21546037003409182
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_logloss: 0.415754	valid_1's binary_logloss: 0.416771
[400]	valid_0's bina

In [14]:
gc.collect()
df_fea, retain_index, ifdrop = test_on_sample(train_csr_f,train_y,lgb_model,n_fold = 5,seed=19,first=2500)

Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_logloss: 0.416003	valid_1's binary_logloss: 0.415773
[400]	valid_0's binary_logloss: 0.413798	valid_1's binary_logloss: 0.41469
[600]	valid_0's binary_logloss: 0.412166	valid_1's binary_logloss: 0.414281
[800]	valid_0's binary_logloss: 0.410742	valid_1's binary_logloss: 0.414103
[1000]	valid_0's binary_logloss: 0.409395	valid_1's binary_logloss: 0.413988
[1200]	valid_0's binary_logloss: 0.408112	valid_1's binary_logloss: 0.41391
[1400]	valid_0's binary_logloss: 0.406889	valid_1's binary_logloss: 0.413861
[1600]	valid_0's binary_logloss: 0.405676	valid_1's binary_logloss: 0.413817
[1800]	valid_0's binary_logloss: 0.404507	valid_1's binary_logloss: 0.413788
[2000]	valid_0's binary_logloss: 0.403356	valid_1's binary_logloss: 0.41377
[2200]	valid_0's binary_logloss: 0.402219	valid_1's binary_logloss: 0.413763
[2400]	valid_0's binary_logloss: 0.401137	valid_1's binary_logloss: 0.413783
Early stopping, best

KeyboardInterrupt: 

In [12]:
gc.collect()
df_fea, retain_index, ifdrop = test_on_sample(train_csr_f,train_y,lgb_model,n_fold = 5,seed=19,first=2500)

Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_logloss: 0.416003	valid_1's binary_logloss: 0.415773
[400]	valid_0's binary_logloss: 0.413798	valid_1's binary_logloss: 0.41469
[600]	valid_0's binary_logloss: 0.412166	valid_1's binary_logloss: 0.414281
Did not meet early stopping. Best iteration is:
[600]	valid_0's binary_logloss: 0.412166	valid_1's binary_logloss: 0.414281
[0.4142813974421431]
cv: 0.4142813974421431
test mean: 0.21772465863943333


KeyboardInterrupt: 

In [27]:
gc.collect()
df_fea, retain_index, ifdrop = test_on_sample(train_csr_f,train_y,lgb_model,n_fold = 5,seed=19,first=2500)

Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_logloss: 0.417182	valid_1's binary_logloss: 0.416848
[400]	valid_0's binary_logloss: 0.415046	valid_1's binary_logloss: 0.415452
[600]	valid_0's binary_logloss: 0.41363	valid_1's binary_logloss: 0.414941
[800]	valid_0's binary_logloss: 0.412472	valid_1's binary_logloss: 0.414657
[1000]	valid_0's binary_logloss: 0.411397	valid_1's binary_logloss: 0.414483
[1200]	valid_0's binary_logloss: 0.410399	valid_1's binary_logloss: 0.41436
[1400]	valid_0's binary_logloss: 0.409469	valid_1's binary_logloss: 0.414288
[1600]	valid_0's binary_logloss: 0.408552	valid_1's binary_logloss: 0.414213
[1800]	valid_0's binary_logloss: 0.407687	valid_1's binary_logloss: 0.414168
[2000]	valid_0's binary_logloss: 0.406831	valid_1's binary_logloss: 0.414118
[2200]	valid_0's binary_logloss: 0.405995	valid_1's binary_logloss: 0.414086
[2400]	valid_0's binary_logloss: 0.405196	valid_1's binary_logloss: 0.414058
[2600]	valid_0's bi

KeyboardInterrupt: 

In [ ]:
aa

In [ ]:
df_fea, retain_index, ifdrop = test_on_sample(train_csr_f,train_y,lgb_model,n_fold = 5,seed=19,first=2500)

In [ ]:
gc.collect()
df_fea, retain_index, ifdrop = test_on_sample(train_csr_f,train_y,lgb_model,n_fold = 5,seed=19,first=2200)

In [ ]:
first=2400
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr_f,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.4144021068865735])

In [ ]:
df_fea, retain_index, ifdrop = test_on_sample(train_csr,train_y,lgb_model,n_fold = 5,seed=19,first=1500)

In [ ]:
first=2500
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

In [ ]:
first=3500
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

In [ ]:
first=4500
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

In [ ]:
first=1000
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

In [ ]:
first=5000
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))
print(len(retain_index))
verify_drop = verify_on_sample(train_csr,train_y,lgb_model,retain_index,n_fold = 5,seed=19,cv_list=[0.41467523398442835])

In [ ]:
first=2500
split_index = df_fea.sort_values('split',ascending=False)['split'].index.values[:first]
gain_index = df_fea.sort_values('gain',ascending=False)['gain'].index.values[:first]
retain_index = list(set(split_index).intersection(set(gain_index)))

train_csr_save=train_csr_f[:,retain_index]
predict_csr_save=predict_csr_f[:,retain_index]

save_csr(train_csr_save,'../../data/features/xgb/train_csr_{}'.format(first))
save_csr(predict_csr_save,'../../data/features/xgb/predict_csr_{}'.format(first))

In [ ]:
df_fea.to_csv('../../data/features/xgb/fea_importance.csv',index=False)